In [ ]:
pip install sanpy

In [ ]:
# API key - qovlkte53ks5zjha_jcn4x4363pnaroe2
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime

In [ ]:
import san
san.ApiConfig.api_key = 'qovlkte53ks5zjha_jcn4x4363pnaroe2'
san.get("projects/all")

In [ ]:
# Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [ ]:
# Limiting only to Cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

In [ ]:
def metriclist(slug):
    metric = "marketcap_usd" 
    fullname = metric + "/" + slug 
    metrics = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1h"
    )
    newname = slug+"_"+metric
    finaldf = metrics.rename(columns={"value": newname})
    return(finaldf)

In [ ]:
# Change metric and slug fullname = metric + slug
# Running through all cryptocurrencies and adding price and daily returns
count = 0
metric_df = pd.DataFrame()
for i in crypto['slug'].tolist():
    holddf = metriclist(i)
    if count == 0:
        metric_df = holddf
    else:
        metric_df = pd.merge(metric_df, holddf, how = "left", on = "datetime")
    count = count+1